<a href="https://colab.research.google.com/github/Saumitra-Shukla/random_face_generator_gan/blob/master/retrain_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf s*

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!cp drive/'My Drive'/machine-learning/face_gan/face_generator00.h5 gace_generator00.h5

In [0]:
!cp drive/'My Drive'/machine-learning/face_gan/face_gan_dis.h5 face_gan_dis.h5

In [0]:
!cp drive/'My Drive'/machine-learning/face_gan/training_data_face_gan.npy training_data_face_gan.npy

In [9]:
from tensorflow import keras
from tensorflow.python.keras import backend as k
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.models import load_model
import numpy as np
from PIL import Image
from tqdm import tqdm
import os

Using TensorFlow backend.


In [0]:
training_data=np.load('training_data_face_gan.npy')

In [0]:
GENERATE_RES=2
GENERATE_SQUARE=32*GENERATE_RES
IMAGE_CHANNELS=3


PREVIEW_ROWS=4
PREVIEW_COLS=7
PREVIEW_MARGIN=16
SAVE_FREQ=100

SEED_SIZE=100

EPOCHS=100000
BATCH_SIZE=32

In [0]:
def save_images(cnt,noise):
  image_array=np.full((PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE + PREVIEW_MARGIN))
                      ,PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE + PREVIEW_MARGIN))
                      ,3),
                      255,dtype=np.uint8)
  generated_images=generator.predict(noise)
  generated_images=0.5 * generated_images + 0.5
  
  image_count=0
  
  for row in range(PREVIEW_ROWS):
    for col in range(PREVIEW_COLS):
      r= row*(GENERATE_SQUARE+16) + PREVIEW_MARGIN
      c= col*(GENERATE_SQUARE+16) + PREVIEW_MARGIN
 

  image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE]= generated_images[image_count] * 255
  image_count+=1
  
  output_path=os.path.join('drive/My Drive/machine-learning/face_gan/photos/photos_new')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
    
  filename=os.path.join(output_path,f"train-{cnt}.png")
  im=Image.fromarray(image_array)
  im.save(filename)

In [0]:
def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    input_image = Input(shape=image_shape)

    validity = model(input_image)

    return Model(input_image, validity)

In [0]:
def build_generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    model.add(Reshape((4,4,256)))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    # Output resolution, additional upsampling
    for i in range(GENERATE_RES):
      model.add(UpSampling2D())
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    input = Input(shape=(seed_size,))
    generated_image = model(input)

    return Model(input,generated_image)

In [0]:
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)
optimizer = Adam(1.5e-4,0.5) # learning rate and momentum adjusted from paper

discriminator=load_model('face_gan_dis.h5')
discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
generator=load_model('gace_generator00.h5')

random_input = Input(shape=(SEED_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input,validity)
combined.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])



In [19]:
y_real = np.ones((BATCH_SIZE,1))
y_fake = np.zeros((BATCH_SIZE,1))

fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))

cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0,training_data.shape[0],BATCH_SIZE)
    x_real = training_data[idx]

    # Generate some images
    seed = np.random.normal(0,1,(BATCH_SIZE,SEED_SIZE))
    x_fake = generator.predict(seed)

    # Train discriminator on real and fake
    discriminator_metric_real = discriminator.train_on_batch(x_real,y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake,y_fake)
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,discriminator_metric_generated)
    
    # Train generator on Calculate losses
    generator_metric = combined.train_on_batch(seed,y_real)
    
    # Time for an update?
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_seed)
        cnt += 1
        print(f"Epoch {epoch}, Discriminator accuarcy: {discriminator_metric[1]}, Generator accuracy: {generator_metric[1]}")
        
#generator.save(os.path.join(DATA_PATH,"face_generator.h5"))

        generator.save("face_generator01.h5")
        
        

           

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0, Discriminator accuarcy: 0.71875, Generator accuracy: 0.09375


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 100, Discriminator accuarcy: 0.671875, Generator accuracy: 0.0
Epoch 200, Discriminator accuarcy: 0.671875, Generator accuracy: 0.0
Epoch 300, Discriminator accuarcy: 0.59375, Generator accuracy: 0.03125
Epoch 400, Discriminator accuarcy: 0.5, Generator accuracy: 0.09375
Epoch 500, Discriminator accuarcy: 0.84375, Generator accuracy: 0.15625
Epoch 600, Discriminator accuarcy: 0.703125, Generator accuracy: 0.21875
Epoch 700, Discriminator accuarcy: 0.609375, Generator accuracy: 0.09375
Epoch 800, Discriminator accuarcy: 0.453125, Generator accuracy: 0.0625
Epoch 900, Discriminator accuarcy: 0.75, Generator accuracy: 0.125
Epoch 1000, Discriminator accuarcy: 0.8125, Generator accuracy: 0.0
Epoch 1100, Discriminator accuarcy: 0.875, Generator accuracy: 0.125
Epoch 1200, Discriminator accuarcy: 0.765625, Generator accuracy: 0.09375
Epoch 1300, Discriminator accuarcy: 0.6875, Generator accuracy: 0.1875
Epoch 1400, Discriminator accuarcy: 0.75, Generator accuracy: 0.125
Epoch 1500, Dis

In [0]:
discriminator.save('face_gan_dis_001.h5')

In [0]:
combined.save('combined.h5')

In [0]:
!cp face_generator01.h5 drive/'My Drive'/machine-learning/face_gan/face_generator002.h5